In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

Unfortunately, data files don't always come ready to go; sometimes, a dataset must be processed to prepare it for analysis. This is called *data cleaning*. In this notebook, you'll practice data cleaning using an (artificially) messy version of the Iris dataset. First, we'll read in the file and take a look.

In [ ]:
df1 = pd.read_csv('data/iris-messy1.csv')

In [ ]:
df1.head()

## Part 1: Manipulating column values

In the preview, something looks wrong with the first column of the dataset—the values include the string "inches"! Because of this, we can't treat this column as a numeric value. To fix this, we can edit the column using the [apply()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) method, which will apply a function to each value in that column.

In [ ]:
df1['sepal_length'] # the syntax df['column_name'] returns that column of the dataframe

We can pass a *lambda function* to the apply method. A lambda function uses the syntax `lambda x: return_val`, where x is the input argument and return_val is what's returned. For example, `lambda x: x+5` will return the input value + 5.

In [ ]:
processed_col = df1['sepal_length'].apply(lambda x: x.replace(" inches","")) # remove the "inches" text
processed_col

By `apply`ing our lambda function to the sepal_length column, we get a new column. Each value from the original column is passed to the lambda function, and the output goes in the new column.

However, we aren't quite done fixing this column yet. Although those values *look* like numbers, the data type in the dataframe is object (see the `dtype: object` at the bottom of the printout)—for now, you can think of this as a string. Eventually, we want to use these values in quantitative analysis, so let's convert them to [floats](https://www.w3schools.com/python/ref_func_float.asp), a numeric data type, using the apply method again.

In [ ]:
# TODO: use the float() function to convert x to a numeric type
float_col = processed_col.apply(...) 
float_col

Now your column should have `dtype: float64`. Great—let's put these cleaned values back into our dataframe. We can do this with the same `df[column_name]` syntax we used earlier:

In [ ]:
df1['sepal_length'] = float_col

In [ ]:
# TODO: take a look at the dataframe to make sure your new column looks good!
...

## Part 2: Manipulating multiple columns

We actually asked two different scientists to go measure iris flowers for us. We've just looked at the measurements from the first scientist in part 1; now, let's take a look at the data from the second scientist.

In [ ]:
df2 = pd.read_csv('data/iris-messy2.csv')

In [ ]:
df2.head()

Oh no, I forgot! The second scientist is from Europe—they must have taken the measurements in *centimeters*! We'll need to convert these measurements to inches so the data can be combined with the first dataset. We'll need to divide all four of the variable columns by the conversion factor (`2.54`). (Note: you can actually do this in pandas with `df['column_name']/2.54`, but we'll use the `apply` method with a lambda function to practice it one more time.)

In [ ]:
# select multiple columns of a pandas dataframe using df[list_of_column_names]
feature_columns = ['sepal_length','sepal_width','petal_length','petal_width']
df2[feature_columns].head()

In [ ]:
# TODO: use the apply function to divide the four feature_columns by 2.54
inches_vals = df2[feature_columns].apply(...)
inches_vals.head()

Great! Now that we have the values in inches, let's change the values in our dataframe. 

In [ ]:
# we can set multiple columns of the dataframe using the same syntax as above
df2[feature_columns] = inches_vals

In [ ]:
# TODO: take a look at the dataframe to make sure it has the values in inches
...

## Part 3: Combining two DataFrames

Now that `df1` and `df2` are both using inches, let's combine them into a single dataframe. To do this, we'll need to use the pandas [concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) function, which joins together multiple dataframes. First, let's check the shape of both dataframes.

In [ ]:
df1.shape

In [ ]:
df2.shape

It looks like the first scientist collected samples of 100 flowers (i.e., there are 100 rows in `df1`) and the second scientist measured 50 flowers (i.e., there are 50 rows in `df2`). Both dataframes have 5 columns (and we saw in the previous sections that those columns are in the same order, with the same names) so they should be easy to combine! Our new dataset should have information about all 150 flowers, with the same columns, so we are expecting its shape to be `(150, 5)`.

In [ ]:
dfs = ... # TODO: create a list containing df1 and df2
df = pd.concat(dfs)

In [ ]:
... # TODO: check if the shape of the combined dataframe is what we expected

Perfect! We now have created a single dataframe object that stores the data from both scientists. We'll reset the index (numbers displayed on the very left side of the dataframe) so we don't have, for example, two rows labeled #1. 

In [ ]:
df = df.reset_index(drop=True)

## Part 4: Checking data quality

Before this dataset is ready to go, let's do some final checks to make sure our data are clean.

To investigate a numerical variable, one tool we can use is a plot called a *histogram*. This type of graph visualizes what values the variable takes on. Let's look at a histogram for sepal length:

In [ ]:
df.hist(column='sepal_length', bins=40)
plt.show()

Here, the x axis represents the sepal length value, and the y axis counts how many rows have a sepal length at that value. It looks like most of the rows of our dataframe have a sepal length between about 5 and 8 inches. However, there is one *outlier*—a point that doesn't look like the rest. This flower has a sepal length of over 50 inches, which sounds pretty unrealistic.

Let's see if we can take a closer look at that outlier. We can use the [sort_values](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) method on our dataframe to sort the data by a particular column. We'll sort it in descending order (with the argument `ascending=False`) so that the largest values are first.

In [ ]:
var_name = ... # TODO: enter the name of the column we want to sort by
df.sort_values(by=var_name, ascending=False)

Weird—it looks like this row has much higher values for all four numeric columns. Let's compute some summary statistics for each column for comparison.

In [ ]:
# TODO: compute summary statistics of the numeric columns (hint: we've done this before!)
...

In comparison to the mean and median (i.e., 50% quantile) of the data, our outlier is about 10 times as high! I called the scientist to investigate and they realized they'd accidentally left off the decimal points in that row (so 5.5 inches became 55, which pandas displays as 55.0).

Looking again at the sorted dataframe, the outlier row is #81. We can use the pandas function [.loc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html) to access specific rows and columns of the DataFrame.

In [ ]:
df.loc[81] # access row 81

In [ ]:
df.loc[81, feature_columns] # access row 81 and the numeric columns

Based on what you've done in the previous sections, update the DataFrame with the corrected values.

In [ ]:
# TODO: divide the incorrect values by 10
df.loc[81,feature_columns] = ...

In a real analysis, we'd want to look carefully at all four numeric features, but for now, let's move on to checking the class variable. This column of the dataset is a *categorical* variable, meaning it can be any value from some pre-defined set of categories (in this case, the three species of flower). We can check if the values in our DataFrame follow this rule using the pandas [unique()](https://pandas.pydata.org/docs/reference/api/pandas.unique.html) function. This function shows all of the different values (in this case, strings) that are present in a given column.

In [ ]:
df['class'].unique()

Uh oh—it looks like we have four different values in this column instead of three. The shorter name "Virginica" was used somewhere, instead of the category name "Iris-virginica" that should be used.

Using what you've done so far, change the "Virginica" value to "Iris-virginica".

In [ ]:
# TODO: use sort_values to find the "Virginica" row (hint: sort_values sorts strings alphabetically)

In [ ]:
# TODO: use .loc to change the value for that row number and the "class" column to the correct value

In [ ]:
# TODO: use the unique() function to check that there are now only three unique values

# Part 5: Saving the dataset

Finally, we finished cleaning the Iris dataset. We can save our finished product as a `.csv` (comma-separated values) file. This is a common file format for storing datasets, and we can easily read it back into a pandas DataFrame later. Pandas has a [to_csv()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) method to do this.

In [ ]:
filename = 'data/iris_clean.csv'
df.to_csv(filename, index=None)